In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/ST/stargan

Mounted at /content/drive
/content/drive/MyDrive/ST/stargan


In [2]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import LambdaLR
import torch.optim as optim

In [3]:
class TSTRClassifier(nn.Module):
    def __init__(self, num_timesteps=128, num_channels=3, num_classes=5):
        super(TSTRClassifier, self).__init__()

        self.conv1 = nn.Conv1d(num_channels, 16, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(16)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(32)
        self.conv3 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(64)
        self.conv4 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn4 = nn.BatchNorm1d(128)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.25)

        self.fc_shared = nn.Linear(num_timesteps * 8, 100)

        self.fc_class = nn.Linear(100, num_classes)

    def forward(self, x):
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.pool(self.relu(self.bn3(self.conv3(x))))
        x = self.pool(self.relu(self.bn4(self.conv4(x))))
        x = x.view(x.size(0), -1)  # Flatten
        x = self.dropout(x)
        x = self.relu(self.fc_shared(x))

        # Final output for class prediction
        class_outputs = self.fc_class(x)
        return class_outputs


def get_df_data(dataset_name, class_idx, num_train_domains):

    # Load the dataset
    with open(f'data/{dataset_name}.pkl', 'rb') as f:
        x, y, k = pickle.load(f)

    with open(f'data/{dataset_name}_fs.pkl', 'rb') as f:
        fs = pickle.load(f)

    # Filter out the samples that are used for finetuning
    x = x[fs == 0]
    y = y[fs == 0]
    k = k[fs == 0]

    x_ = x[(y == class_idx) & (k < num_train_domains)]
    y_ = y[(y == class_idx) & (k < num_train_domains)]
    k_ = k[(y == class_idx) & (k < num_train_domains)]

    return x_, y_, k_


def get_dp_data(dataset_name, class_idx, num_train_domains):

    # Load the dataset
    with open(f'data/{dataset_name}.pkl', 'rb') as f:
        x, y, k = pickle.load(f)

    with open(f'data/{dataset_name}_fs.pkl', 'rb') as f:
        fs = pickle.load(f)

    # Filter out the samples that are used for finetuning
    x = x[fs == 0]
    y = y[fs == 0]
    k = k[fs == 0]

    x_ = x[(y == class_idx) & (k >= num_train_domains)]
    y_ = y[(y == class_idx) & (k >= num_train_domains)]
    k_ = k[(y == class_idx) & (k >= num_train_domains)] - num_train_domains

    return x_, y_, k_


def remap_labels(y):
    label_map = {clss: i for i, clss in enumerate(np.unique(y))}
    return np.array([label_map[clss] for clss in y])


def setup_training(x_tr, y_tr, x_val, y_val, x_test, y_test, batch_size=64):
    # Convert numpy arrays to torch tensors
    x_train_tensor = torch.tensor(x_tr, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_tr, dtype=torch.long)
    x_val_tensor = torch.tensor(x_val, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val, dtype=torch.long)
    x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)

    # Create datasets and loaders
    train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader


def train_model(model, train_loader, val_loader, loss_fn, optimizer, epochs=300):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    loss_train = []
    loss_val = []
    accuracy_val = []
    best_loss = np.inf
    best_accuracy = 0

    # Set up linear learning rate decay
    lambda_lr = lambda epoch: 1 - epoch / epochs
    scheduler = LambdaLR(optimizer, lr_lambda=lambda_lr)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for x_batch, y_batch in train_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(x_batch)
            loss = loss_fn(outputs, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        total_loss /= len(train_loader)
        loss_train.append(total_loss)

        # Update learning rate
        scheduler.step()

        val_accuracy, val_loss = evaluate_model(model, val_loader, loss_fn)
        if val_accuracy > best_accuracy:
            best_epoch = epoch
            best_accuracy = val_accuracy
            best_loss = val_loss
            best_model_state = model.state_dict().copy()
        loss_val.append(val_loss)
        accuracy_val.append(val_accuracy)

        current_lr = scheduler.get_last_lr()[0]
        print(f"Epoch {epoch + 1}/{epochs} - Train loss: {total_loss:.4f} - Val loss: {val_loss:.4f} - Val accuracy: {val_accuracy:.4f} - LR: {current_lr:.6f}")

    print(f"\nBest epoch: {best_epoch + 1} - Best val accuracy: {best_accuracy:.4f} - Best val loss: {best_loss:.4f}\n")

    return best_model_state


def evaluate_model(model, test_loader, loss_fn):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            outputs = model(x_batch)
            loss = loss_fn(outputs, y_batch)
            total_loss += loss.item()

            _, predicted_labels = torch.max(outputs, 1)
            correct_predictions += (predicted_labels == y_batch).sum().item()
            total_predictions += len(y_batch)

    total_loss /= len(test_loader)
    accuracy = correct_predictions / total_predictions

    return accuracy, total_loss


def calculate_tstr_score(x_train, y_train, x_test, y_test):
    # Remap labels
    y_train = remap_labels(y_train)
    y_test = remap_labels(y_test)

    x_tr, x_val, y_tr, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=2710, stratify=y_train, shuffle=True)
    tr_loader, val_loader, test_loader = setup_training(x_tr, y_tr, x_val, y_val, x_test, y_test, batch_size=64)

    model = TSTRClassifier(num_timesteps=x_train.shape[2], num_channels=x_train.shape[1], num_classes=len(np.unique(y_train)))
    loss_fn = nn.CrossEntropyLoss()
    initial_lr = 0.0001
    optimizer = optim.Adam(model.parameters(), lr=initial_lr)

    best_model_state = train_model(model, tr_loader, val_loader, loss_fn, optimizer, epochs=50)
    best_model = TSTRClassifier(num_timesteps=x_train.shape[2], num_channels=x_train.shape[1], num_classes=len(np.unique(y_train)))
    best_model.load_state_dict(best_model_state)
    test_accuracy, test_loss = evaluate_model(best_model, test_loader, loss_fn)

    return test_accuracy, test_loss

In [4]:
dataset = 'realworld'

if dataset == 'realworld':
    dataset_name = 'realworld_128_3ch_4cl'
    num_df_domains = 10
    num_dp_domains = 5
    num_classes = 4
    class_names = ['WAL', 'RUN', 'CLD', 'CLU']

elif dataset == 'cwru':
    dataset_name = 'cwru_256_3ch_5cl'
    num_df_domains = 4
    num_dp_domains = 4
    num_classes = 5
    class_names = ['IR', 'Ball', 'OR_centred', 'OR_orthogonal', 'OR_opposite']

classes_dict = {clss: i for i, clss in enumerate(class_names)}

accs = []

for src_class in class_names:
    trg_classes = [clss for clss in class_names if clss != src_class]

    x_df = []
    y_df = []
    k_df = []

    x_dp = []
    y_dp = []
    k_dp = []

    for trg_class in trg_classes:
        x_df_, y_df_, k_df_ = get_df_data(dataset_name, classes_dict[trg_class], num_df_domains)
        x_dp_, y_dp_, k_dp_ = get_dp_data(dataset_name, classes_dict[trg_class], num_df_domains)

        x_df.append(x_df_)
        y_df.append(y_df_)
        k_df.append(k_df_)

        x_dp.append(x_dp_)
        y_dp.append(y_dp_)
        k_dp.append(k_dp_)

    x_df = np.concatenate(x_df, axis=0)
    y_df = np.concatenate(y_df, axis=0)
    k_df = np.concatenate(k_df, axis=0)

    x_dp = np.concatenate(x_dp, axis=0)
    y_dp = np.concatenate(y_dp, axis=0)
    k_dp = np.concatenate(k_dp, axis=0)

    acc, loss = calculate_tstr_score(x_df, y_df, x_dp, y_dp)
    accs.append(acc)

    print(f'{src_class} -> {trg_classes}: {acc:.2f}\n\n')

print(f'Mean accuracy: {np.mean(accs):.2f}')

Epoch 1/50 - Train loss: 0.5332 - Val loss: 0.2753 - Val accuracy: 0.9178 - LR: 0.000098
Epoch 2/50 - Train loss: 0.2086 - Val loss: 0.1505 - Val accuracy: 0.9527 - LR: 0.000096
Epoch 3/50 - Train loss: 0.1597 - Val loss: 0.1134 - Val accuracy: 0.9575 - LR: 0.000094
Epoch 4/50 - Train loss: 0.1048 - Val loss: 0.0993 - Val accuracy: 0.9679 - LR: 0.000092
Epoch 5/50 - Train loss: 0.0897 - Val loss: 0.0916 - Val accuracy: 0.9631 - LR: 0.000090
Epoch 6/50 - Train loss: 0.0786 - Val loss: 0.0828 - Val accuracy: 0.9688 - LR: 0.000088
Epoch 7/50 - Train loss: 0.0681 - Val loss: 0.0822 - Val accuracy: 0.9669 - LR: 0.000086
Epoch 8/50 - Train loss: 0.0611 - Val loss: 0.0758 - Val accuracy: 0.9716 - LR: 0.000084
Epoch 9/50 - Train loss: 0.0526 - Val loss: 0.0706 - Val accuracy: 0.9745 - LR: 0.000082
Epoch 10/50 - Train loss: 0.0568 - Val loss: 0.0664 - Val accuracy: 0.9735 - LR: 0.000080
Epoch 11/50 - Train loss: 0.0455 - Val loss: 0.0661 - Val accuracy: 0.9726 - LR: 0.000078
Epoch 12/50 - Train

In [5]:
dataset = 'cwru'

if dataset == 'realworld':
    dataset_name = 'realworld_128_3ch_4cl'
    num_df_domains = 10
    num_dp_domains = 5
    num_classes = 4
    class_names = ['WAL', 'RUN', 'CLD', 'CLU']

elif dataset == 'cwru':
    dataset_name = 'cwru_256_3ch_5cl'
    num_df_domains = 4
    num_dp_domains = 4
    num_classes = 5
    class_names = ['IR', 'Ball', 'OR_centred', 'OR_orthogonal', 'OR_opposite']

classes_dict = {clss: i for i, clss in enumerate(class_names)}

accs = []

for src_class in class_names:
    trg_classes = [clss for clss in class_names if clss != src_class]

    x_df = []
    y_df = []
    k_df = []

    x_dp = []
    y_dp = []
    k_dp = []

    for trg_class in trg_classes:
        x_df_, y_df_, k_df_ = get_df_data(dataset_name, classes_dict[trg_class], num_df_domains)
        x_dp_, y_dp_, k_dp_ = get_dp_data(dataset_name, classes_dict[trg_class], num_df_domains)

        x_df.append(x_df_)
        y_df.append(y_df_)
        k_df.append(k_df_)

        x_dp.append(x_dp_)
        y_dp.append(y_dp_)
        k_dp.append(k_dp_)

    x_df = np.concatenate(x_df, axis=0)
    y_df = np.concatenate(y_df, axis=0)
    k_df = np.concatenate(k_df, axis=0)

    x_dp = np.concatenate(x_dp, axis=0)
    y_dp = np.concatenate(y_dp, axis=0)
    k_dp = np.concatenate(k_dp, axis=0)

    acc, loss = calculate_tstr_score(x_df, y_df, x_dp, y_dp)
    accs.append(acc)

    print(f'{src_class} -> {trg_classes}: {acc:.2f}\n\n')

print(f'Mean accuracy: {np.mean(accs):.2f}')

Epoch 1/50 - Train loss: 0.4808 - Val loss: 0.3004 - Val accuracy: 0.8123 - LR: 0.000098
Epoch 2/50 - Train loss: 0.0928 - Val loss: 1.3312 - Val accuracy: 0.7354 - LR: 0.000096
Epoch 3/50 - Train loss: 0.0282 - Val loss: 0.2709 - Val accuracy: 0.8760 - LR: 0.000094
Epoch 4/50 - Train loss: 0.0148 - Val loss: 0.0969 - Val accuracy: 0.9602 - LR: 0.000092
Epoch 5/50 - Train loss: 0.0135 - Val loss: 0.2875 - Val accuracy: 0.8840 - LR: 0.000090
Epoch 6/50 - Train loss: 0.0090 - Val loss: 0.0612 - Val accuracy: 0.9748 - LR: 0.000088
Epoch 7/50 - Train loss: 0.0057 - Val loss: 0.0342 - Val accuracy: 0.9887 - LR: 0.000086
Epoch 8/50 - Train loss: 0.0047 - Val loss: 0.0023 - Val accuracy: 1.0000 - LR: 0.000084
Epoch 9/50 - Train loss: 0.0045 - Val loss: 0.0029 - Val accuracy: 0.9993 - LR: 0.000082
Epoch 10/50 - Train loss: 0.0050 - Val loss: 0.0276 - Val accuracy: 0.9920 - LR: 0.000080
Epoch 11/50 - Train loss: 0.0037 - Val loss: 0.0015 - Val accuracy: 1.0000 - LR: 0.000078
Epoch 12/50 - Train